# SWRL Rules Creation for German Tourism Ontology

This notebook implements 4 SWRL rules that infer new class memberships based on existing properties:

1. **BudgetFriendlyActivity**: Activities with `budget_free` or `budget_low`
2. **RainyDayOption**: Activities with `location_indoor` setting
3. **EnglishFriendlyTour**: Tours available in English
4. **FamilyFriendlyActivity**: Parks (which are generally family-friendly)

In [ ]:
from pathlib import Path
from owlready2 import *
import os

In [ ]:
# Define paths
BASE_DIR = Path(os.getcwd()).parent
ONTOLOGY_PATH = BASE_DIR / "ontologies" / "populated" / "german_city_tourism_populated.owl"
OUTPUT_PATH = BASE_DIR / "ontologies" / "populated" / "german_city_tourism_with_rules.owl"

print(f"Loading ontology from: {ONTOLOGY_PATH}")

In [ ]:
# Load the ontology
onto = get_ontology(str(ONTOLOGY_PATH)).load()

In [ ]:
# Create new inferred classes that our SWRL rules will populate
with onto:
    class BudgetFriendlyActivity(onto.Activity):
        """An activity that is free or low-cost, suitable for budget-conscious travelers."""
        pass
    
    class RainyDayOption(onto.Activity):
        """An indoor activity suitable for rainy or bad weather days."""
        pass
    
    class EnglishFriendlyTour(onto.Tour):
        """A tour that is available in English, accessible for international tourists."""
        pass
    
    class FamilyFriendlyActivity(onto.Activity):
        """An activity suitable for families with children."""
        pass

print("Created inferred classes:")
print(f"  - BudgetFriendlyActivity (subclass of Activity)")
print(f"  - RainyDayOption (subclass of Activity)")
print(f"  - EnglishFriendlyTour (subclass of Tour)")
print(f"  - FamilyFriendlyActivity (subclass of Activity)")

In [ ]:
# Get the specific instances we need for our rules
budget_free = onto.budget_free
budget_low = onto.budget_low
location_indoor = onto.location_indoor
lang_english = onto.lang_english

print("Retrieved instances:")
print(f"  - budget_free: {budget_free}")
print(f"  - budget_low: {budget_low}")
print(f"  - location_indoor: {location_indoor}")
print(f"  - lang_english: {lang_english}")

In [ ]:
# Define SWRL rules using Owlready2's Imp (implies) class
with onto:
    # Rule 1a: Activity(?a) ^ hasBudget(?a, budget_free) -> BudgetFriendlyActivity(?a)
    rule1a = Imp()
    rule1a.set_as_rule(
        "Activity(?a), hasBudget(?a, budget_free) -> BudgetFriendlyActivity(?a)"
    )
    
    # Rule 1b: Activity(?a) ^ hasBudget(?a, budget_low) -> BudgetFriendlyActivity(?a)
    rule1b = Imp()
    rule1b.set_as_rule(
        "Activity(?a), hasBudget(?a, budget_low) -> BudgetFriendlyActivity(?a)"
    )
    
    # Rule 2: Activity(?a) ^ hasLocationSetting(?a, location_indoor) -> RainyDayOption(?a)
    rule2 = Imp()
    rule2.set_as_rule(
        "Activity(?a), hasLocationSetting(?a, location_indoor) -> RainyDayOption(?a)"
    )
    
    # Rule 3: Tour(?t) ^ hasLanguage(?t, lang_english) -> EnglishFriendlyTour(?t)
    rule3 = Imp()
    rule3.set_as_rule(
        "Tour(?t), hasLanguage(?t, lang_english) -> EnglishFriendlyTour(?t)"
    )
    
    # Rule 4: Park(?p) -> FamilyFriendlyActivity(?p)
    rule4 = Imp()
    rule4.set_as_rule(
        "Park(?p) -> FamilyFriendlyActivity(?p)"
    )

print("Defined 5 SWRL rules (4 logical rules, Rule 1 split into 1a and 1b):")
print("  Rule 1a: Activity(?a), hasBudget(?a, budget_free) -> BudgetFriendlyActivity(?a)")
print("  Rule 1b: Activity(?a), hasBudget(?a, budget_low) -> BudgetFriendlyActivity(?a)")
print("  Rule 2:  Activity(?a), hasLocationSetting(?a, location_indoor) -> RainyDayOption(?a)")
print("  Rule 3:  Tour(?t), hasLanguage(?t, lang_english) -> EnglishFriendlyTour(?t)")
print("  Rule 4:  Park(?p) -> FamilyFriendlyActivity(?p)")

In [ ]:
# Run the Pellet reasoner (supports SWRL rules better than HermiT)
# Note: Make sure you have Java installed and Pellet available
print("Running reasoner to apply SWRL rules...")
try:
    sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True)
    print("Pellet reasoner completed successfully!")
except Exception as e:
    print(f"Pellet failed: {e}")
    print("\nTrying HermiT reasoner (limited SWRL support)...")
    sync_reasoner_hermit()
    print("HermiT reasoner completed.")

In [ ]:
# Verify the rule inferences
print("=" * 60)
print("VERIFICATION OF SWRL RULE INFERENCES")
print("=" * 60)

# Count BudgetFriendlyActivity instances
budget_friendly = list(onto.BudgetFriendlyActivity.instances())
print(f"\n1. BudgetFriendlyActivity instances: {len(budget_friendly)}")
if budget_friendly:
    print(f"   Sample: {[str(x.name) for x in budget_friendly[:5]]}...")

# Count RainyDayOption instances
rainy_day = list(onto.RainyDayOption.instances())
print(f"\n2. RainyDayOption instances: {len(rainy_day)}")
if rainy_day:
    print(f"   Sample: {[str(x.name) for x in rainy_day[:5]]}...")

# Count EnglishFriendlyTour instances
english_tours = list(onto.EnglishFriendlyTour.instances())
print(f"\n3. EnglishFriendlyTour instances: {len(english_tours)}")
if english_tours:
    print(f"   Sample: {[str(x.name) for x in english_tours[:5]]}...")

# Count FamilyFriendlyActivity instances
family_friendly = list(onto.FamilyFriendlyActivity.instances())
print(f"\n4. FamilyFriendlyActivity instances: {len(family_friendly)}")
if family_friendly:
    print(f"   Sample: {[str(x.name) for x in family_friendly[:5]]}...")

print("\n" + "=" * 60)

In [ ]:
# Save the ontology with the new rules
onto.save(file=str(OUTPUT_PATH), format="rdfxml")
print(f"Ontology with SWRL rules saved to: {OUTPUT_PATH}")

## Summary of SWRL Rules

| # | Rule Name | SWRL Syntax | Purpose |
|---|-----------|-------------|----------|
| 1a | BudgetFriendlyActivity (Free) | `Activity(?a), hasBudget(?a, budget_free) -> BudgetFriendlyActivity(?a)` | Classify free activities |
| 1b | BudgetFriendlyActivity (Low) | `Activity(?a), hasBudget(?a, budget_low) -> BudgetFriendlyActivity(?a)` | Classify low-cost activities |
| 2 | RainyDayOption | `Activity(?a), hasLocationSetting(?a, location_indoor) -> RainyDayOption(?a)` | Weather-based recommendations |
| 3 | EnglishFriendlyTour | `Tour(?t), hasLanguage(?t, lang_english) -> EnglishFriendlyTour(?t)` | International tourist accessibility |
| 4 | FamilyFriendlyActivity | `Park(?p) -> FamilyFriendlyActivity(?p)` | Family-suitable venues |

### Use Cases

- **Budget Planning**: Query `BudgetFriendlyActivity` to find affordable options
- **Weather Adaptation**: Query `RainyDayOption` when weather is bad
- **Tourist Accessibility**: Query `EnglishFriendlyTour` for non-German speakers
- **Family Travel**: Query `FamilyFriendlyActivity` for family-friendly activities